# Great Energy Predictor - Preprocessing
#### Hosted by: ASHRAE
##### Source: https://www.kaggle.com/c/ashrae-energy-prediction

## Section I: Dependencies and Data

### Dependencies

In [1]:
import os
import gc
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import src.utils as udf

In [2]:
# Plot settings
sns.set(rc={'figure.figsize': (16, 4),
            'font.size': 12})

### Data

In [3]:
# Data path
data_path = '../data/from_eda/'

In [4]:
# Meter data
meter = pd.read_pickle(f'{data_path}meter.pkl')
meter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20138871 entries, 0 to 20216099
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    uint16        
 1   meter          uint8         
 2   timestamp      datetime64[ns]
 3   meter_reading  float32       
 4   site_id        uint8         
 5   meter_type     object        
 6   dayofyear      uint16        
 7   month          uint8         
 8   day            uint8         
 9   dayofweek      uint8         
 10  hour           uint8         
dtypes: datetime64[ns](1), float32(1), object(1), uint16(2), uint8(6)
memory usage: 729.8+ MB


In [5]:
# Weather data
weather = pd.read_pickle(f'{data_path}weather.pkl')
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   site_id             139773 non-null  uint8         
 1   timestamp           139773 non-null  datetime64[ns]
 2   air_temperature     139718 non-null  float32       
 3   cloud_coverage      70600 non-null   float32       
 4   dew_temperature     139660 non-null  float32       
 5   precip_depth_1_hr   89484 non-null   float32       
 6   sea_level_pressure  129155 non-null  float32       
 7   wind_direction      133505 non-null  float32       
 8   wind_speed          139469 non-null  float32       
dtypes: datetime64[ns](1), float32(7), uint8(1)
memory usage: 4.9 MB


In [6]:
# Building data
building = pd.read_pickle(f'{data_path}building.pkl')
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   uint8  
 1   building_id  1449 non-null   uint16 
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   uint32 
 4   year_built   675 non-null    float32
 5   floor_count  355 non-null    float32
dtypes: float32(2), object(1), uint16(1), uint32(1), uint8(1)
memory usage: 32.7+ KB


In [7]:
del data_path
gc.collect()

77

## Section II: Meter Preprocessing

### Missing values and zero values in meter data

Aside from the missing meter readings, there are a lot of zero-readings. In the last notebook (`01-eda.ipynb`), we saw that site 0 buildings had readings of 0 for the first 5 months. It is a lot more likely that zero-readings are errors in the data than actually readings of 0, so we will inspect the readings.

In [ ]:
# Visualize missing values and zeroes
# Adapted from: https://www.kaggle.com/hmendonca/clean-weather-data-eda#Check-the-meter-averages-per-weekday

meter_types = ['electricity', 'chilledwater', 'steam', 'hotwater']
fig, ax = plt.subplots(1, 4, figsize=(20, 30))

for m in range(4):
    
    df = meter[meter.meter == m]
    mmap = df.pivot(index='building_id', columns='timestamp', values='meter_reading')
    mmap = mmap.reindex(sorted(meter.building_id.unique()))
    mmap = np.sign(mmap)
    
    sns.heatmap(mmap, cmap='Paired', ax=ax[m], cbar=False)
    ax[m].set_title(f'{meter_types[m]} meter')
    ax[m].set_ylabel(None)
    ax[m].set_xlabel(None)

This code was adapted from a Kaggle user (source in the comment above). For clarity, the plot description is as follows:
- Each of the 4 subplots represents the meter readings from a particular meter type
    - From left to right: `electricity`, `chilledwater`, `steam`, `hotwater`
- Y-axis: building number
- X-axis: 2016 timestamp
- Orange represents non-zero meter readings
- Blue represents zero-readings
- No color represents no readings

Patterns in the meter readings:
- Site 0 buildings are missing readings for both `electricity` and `chilledwater` meters at the beginning of the year
- Site 15 buildings are missing a chunk of readings between February and March for all their meters: `electricity`, `chilledwater`, and `steam`
- A lot of the blue shading come in streaks of long periods of time, so these may actually be missing readings

The most efficient course of action here would be to just remove these zero-readings.

In [ ]:
# Drop observations with a meter reading of 0
meter.drop(index=meter[meter.meter_reading <= 0].index, inplace=True)
meter.shape

### Buildings built in 2016 and 2017

The data contains buildings with a `year_built` value of 2016 and 2017. This is strange as the meter data contains readings from 2016. It's possible that the readings from these buildings are actually the energy consumption during construction. Either way, these buildings are not aligned with our purpose of predicting energy consumption in normal conditions, so we will inspect these buildings and most likely drop them from our data.

In [ ]:
# Inspect buildings with a year_built of 2016 and 2017
yr_built_16_17 = building[building.year_built.isin([2016, 2017])]
yr_built_16_17

In [ ]:
# Count meter readings in these buildings
n_readings_16_17 = meter[meter.building_id.isin(yr_built_16_17.building_id)] # Filter for 2016/2017 buildings
n_readings_16_17 = n_readings_16_17.groupby('building_id').meter_reading.count() # Count readings for each building
n_readings_16_17['Total'] = n_readings_16_17.sum() # Calculate total
pd.DataFrame(n_readings_16_17).reset_index() # Convert to df

There are almost 60 thousand meter readings from buildings built in either 2016 or 2017. That's quite a bit of meter readings, but relatively insignificant when we consider that there are over 18 million readings in the data. We will drop these buildings and their meter readings from the data.

In [ ]:
# Remove 2016 and 2017 buildings
meter = meter[~meter.building_id.isin(yr_built_16_17.building_id)]
meter.shape

In [ ]:
del meter_types, fig, ax, m, df, mmap, yr_built_16_17, n_readings_16_17
gc.collect()

## Section III: Weather Preprocessing

### Reindex weather data

In the last notebook, we saw that the weather data for 2016 was incomplete, i.e. it did not have a record for every hour of every day in every site (24 hours, 366 days in 2016, 16 sites). In other words, there should be 24 x 366 x 16, or `140,544`, timestamps, but more than 700 were missing. So we will reindex the weather data to include every timestamp.

In [ ]:
# Reindex weather data to include every hour of every day in 2016
weather = udf.reidx_site_time(weather, t_start='1/1/2016 00:00:00', t_end='12/31/2016 23:00:00')
weather.info()

### Inspect missing values

In [ ]:
# Missing values
wthr_missing = pd.DataFrame(zip(weather.isnull().sum(), np.round(weather.isnull().mean() * 100, 2)), 
                            index=weather.columns, columns=['missing', 'pct_missing'])
wthr_missing

In [ ]:
# Missing values by site
udf.missing_vals_by_site(weather, pct=True)

There are a lot of missing `cloud_coverage` and `precip_depth_1_hr` values in the data, with a couple of sites even missing 100% of these values. Filling in this much data may introduce a large bias to the data, so we will be dropping these weather features instead.

All of the other weather features have less than 10% of their values missing, so filling their values wouldn't have as much of an adverse effect.

In [ ]:
# Drop cloud_coverage and precip_depth_1_hr
weather.drop(['cloud_coverage', 'precip_depth_1_hr'], axis=1, inplace=True)
weather.head(3)

### Interpolate missing data

In [ ]:
# Time interval to plot
start, end = datetime.date(2016, 1, 1), datetime.date(2016, 1, 31)

# Site 15 air temperature and wind speed
fig, ax = plt.subplots(2, 1, figsize=(16, 8))
w15 = udf.get_site(weather, 15, time_idx=True)
w15.loc[start:end, ['air_temperature', 'wind_speed']].plot(subplots=True, ax=ax, xticks=[], xlabel='Day', 
                                                           title='January 2016 Weather (with missing values)')
plt.show()

In [ ]:
# Use linear interpolation for variables with min/max constraints and cubic interpolation for the rest
lin_cols = ['wind_direction', 'wind_speed']
cub_cols = ['air_temperature', 'dew_temperature', 'sea_level_pressure']

# Fill missing values by site
weather = udf.fill_missing(weather, [], lin_cols, cub_cols)
udf.missing_vals_by_site(weather, pct=True)

In [ ]:
# Site 15 air temperature and wind speed
fig, ax = plt.subplots(2, 1, figsize=(16, 8))
w15 = udf.get_site(weather, 15, time_idx=True)
w15.loc[start:end, ['air_temperature', 'wind_speed']].plot(subplots=True, ax=ax, xticks=[], xlabel='Day', 
                                                           title='January 2016 Weather')
plt.show()

##### Cubic interpolation
I decided to use cubic interpolation because using a polynomial higher than 1 would result in a much smoother estimation of the missing data, as seen in the plot above. Of course, this wasn't possible for every feature due to value constraints, so it was only used for `air_temperature`, `dew_temperature`, and `sea_level_pressure`.

##### Linear interpolation
Since `wind_direction` is limited to values between 0 and 360, and `wind_speed` is limited to non-negative values, linear interpolation was a better choice because it wouldn't result in values outside of these constraints.

##### Forward/backward fill
As interpolation only works for missing values between non-missing values, any leading or trailing missing values (at the beginning or end of the year) was filled using a forward/backward fill.

### Fill missing `sea_level_pressure` values in site 5

The data imputation used above was done on a per-site basis, which means if a site was missing all of its values for a certain feature, this method wouldn't work. While we did manage to fill in most of the missing weather data, site 5 was missing 100% of the `sea_level_pressure` values so these values remain missing.

From the EDA in the last notebook, we discovered that site 5 is in Europe. Because sites 1 and 12 are also in Europe, they may share similar weather to site 5. Without much other context, copying the `sea_level_pressure` from either site 1 or 12 over to site 5 may be our best estimation for the missing values.

In [ ]:
# Extract the 3 European sites
w1 = udf.get_site(weather, 1, time_idx=True)
w5 = udf.get_site(weather, 5, time_idx=True)
w12 = udf.get_site(weather, 12, time_idx=True)

# Air temperature in January from all 3
w1.loc[start:end, 'air_temperature'].plot(label='Site 1')
w5.loc[start:end, 'air_temperature'].plot(label='Site 5')
w12.loc[start:end, 'air_temperature'].plot(label='Site 12')

# Formatting
plt.title('January 2016 Temperature of European Sites')
plt.ylabel('Air temperature')
plt.xlabel('Day')
plt.legend()
plt.show()

In [ ]:
# Sea-level pressure in January from sites 
w1.loc[start:end, 'sea_level_pressure'].plot(label='Site 1')
w12.loc[start:end, 'sea_level_pressure'].plot(label='Site 12')
plt.title('January 2016 Sea-level Pressure of Sites 1 and 12')
plt.ylabel('Sea level pressure')
plt.xlabel('Day')
plt.legend()
plt.show()

It looks like the weather of site 5 is more similar to site 1 than site 12, so we will be using the `sea_level_pressure` from site 1 to copy over to site 5.

In [ ]:
# Indices of sites 1 and 5
i1 = weather[weather.site_id == 1].index 
i5 = weather[weather.site_id == 5].index

# Copy site 1's sea_level_pressure data over to site 5
weather.loc[i5, 'sea_level_pressure'] = weather.loc[i1, 'sea_level_pressure'].values
weather[weather.site_id == 5].head(3)

In [ ]:
# Inspect missing values again
weather.isnull().sum()

### Recast wind direction to integer type

In [ ]:
# Convert wind_direction to unsigned integers
weather['wind_direction'] = weather.wind_direction.astype('uint16')
weather.info()

In [ ]:
del wthr_missing, start, end, w15, lin_cols, cub_cols, w1, w5, w12, i1, i5, fig, ax
gc.collect()

## Section IV: Building Preprocessing

### Missing values

In [ ]:
# Missing values
bldg_missing = pd.DataFrame(zip(building.isnull().sum(), round(building.isnull().mean() * 100, 2)), 
                            index=building.columns, columns=['missing', 'pct_missing'])
bldg_missing

In [ ]:
# Check feature correlation
building.corr()

Even with more than 75% of its values missing, `floor_count` has a moderate linear correlation with `square_feet`. It certainly makes sense, since both of these features are describing the size of the building. We will drop `floor_count` because of the redundancy and since it's missing most of its values anyway.

`Year_built`, on the other hand, does seem to be providing information that the other features are not, so it may be useful to keep this feature. We will be imputing the missing `year_built` values with the median because it's more intuitive to fill a year value with an integer statistic and the median is more robust to outliers than the mean. 

Instead of using the overall median, each building's missing `year_built` value will be imputed with its site's median `year_built`. This would be a better estimation since buildings in the same area are likely to have been built around the same time. But imputing more than half the data means we're guessing on quite a bit of data, so let's first create a boolean missing indicator for `year_built` so that we'll know downstream which values were imputed.

In [ ]:
# Drop floor_count
building.drop('floor_count', axis=1, inplace=True)

# Add missing indicator for year_built
building['missing_year'] = building.year_built.isnull().astype('uint8')
building.head(3)

In [ ]:
# Number of buildings missing year by site
yr_missing_by_site = pd.DataFrame()
yr_missing_by_site['n_values'] = building.groupby('site_id').building_id.count().values
yr_missing_by_site['n_missing'] = building.groupby('site_id').missing_year.sum().values
yr_missing_by_site['pct_missing'] = (yr_missing_by_site.n_missing / yr_missing_by_site.n_values).round(4) * 100
yr_missing_by_site

In [ ]:
# Sites with at least 1 recorded year_built value
sites_with_yr_vals = yr_missing_by_site[yr_missing_by_site.pct_missing < 100].index
i = 0

# Plot these sites
fig, ax = plt.subplots(2, 4, figsize=(15, 6))
fig.subplots_adjust(hspace=0.3)
for a in ax:
    for b in range(4):
        bldg = building[building.site_id == sites_with_yr_vals[i]]
        a[b].hist(bldg.year_built)
        a[b].set_title('Site {} ({} buildings recorded)'.format(sites_with_yr_vals[i], bldg.shape[0]))
        i += 1

Half of the sites are missing all of their `year_built` values so the original plan of imputing missing values with site medians won't be able to fill all of the missing data. Let's revise the imputation method for `year_built` a little bit:
1. Impute missing values with the site medians
2. For sites missing 100% of its values, impute the missing values with the median of similar `primary_use` buildings
    - For example, an education building, missing its `year_built` value, will be filled with the median `year_built` of all other education buildings
3. If there are still missing values, impute with the site median again
4. If there are still missing values, impute with the `primary_use` median again

In [ ]:
# Fill missing year_built values with the building's site's median `year_built` value
building['year_built'] = building.groupby('site_id').year_built.transform(lambda y: y.fillna(y.median()))

# Check percent missing `year_built` again
(yr_missing_by_site['n_values'] - building.groupby('site_id').year_built.count()) * 100 / \
yr_missing_by_site['n_values']

In [ ]:
# Fill the other sites with primary_use medians
building['year_built'] = building.groupby('primary_use').year_built.transform(lambda y: y.fillna(y.median()))

# Fill the rest with the building's site's median
building['year_built'] = building.groupby('site_id').year_built.transform(lambda y: y.fillna(y.median())).astype(int)

# Check null count again
building.isnull().sum()

In [ ]:
# Distribution of year-built values
building.year_built.plot.hist(bins=40, title='Distribution of year_built')
plt.show()

In [ ]:
del bldg_missing, yr_missing_by_site, sites_with_yr_vals, i, fig, ax, a, b, bldg
gc.collect()

Since we did impute missing values using the median, most of the values will inevitably be concentrated somewhere around the middle as seen in the barchart above. However, we did manage to spread out the imputation a little bit by grouping by site for some and `primary_use` for others. Otherwise, there would be 1 tall bar in the middle, instead of the 3 above.

## Section V: Preprocessing Conclusion

### Save dataframes

In [ ]:
# Create new subdirectory in data directory
output_path = '../data/from_prep/'
udf.mkdir(output_path)

In [ ]:
# Save meter data
meter.to_pickle(output_path + 'meter.pkl')
pd.read_pickle(output_path + 'meter.pkl').head(2)

In [ ]:
# Save weather data
weather.to_pickle(output_path + 'weather.pkl')
pd.read_pickle(output_path + 'weather.pkl').head(2)

In [ ]:
# Save building data
building.to_pickle(output_path + 'building.pkl')
pd.read_pickle(output_path + 'building.pkl').head(2)